## Aplicación a caso práctico real del paquete recore 

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('com.csv')
df

,component,molar_percentage,molar_mass,density
0,7,2.87,96.0,0.738
1,8,4.08,107.0,0.765
2,9,3.51,121.0,0.781
3,10,3.26,134.0,0.792
4,11,2.51,147.0,0.796
5,12,2.24,161.0,0.810
6,13,2.18,175.0,0.825
7,14,2.07,190.0,0.836
8,15,2.03,206.0,0.842
9,16,1.67,222.0,0.849


In [3]:
carbon_number = df['component'].to_numpy()
molar_fraction = df['molar_percentage'].to_numpy()
molecular_weight = df['molar_mass'].to_numpy()
density = df['density'].to_numpy()

carbonnumber_max = carbon_number[-2]
molar_fraction_max = molar_fraction[-2]
molecular_weight_max = molecular_weight[-2]
density_max = density[-2]

Primero ajusto para obtener todas las constantes intervinientes. Para eso utilizo la librería Foo_fit.

Dato: Si no quisiera obtener o chequear los valores de los parámetros de ajuste puedo pasar directamente a la distribución y al no darle a la función un valor para las constantes las calcula automáticamente a partir de un diccionario definido en la clase.

En este ejemplo, por cuestiones de evaluación se calculan y se evidencian

In [4]:
from recore import Foo_fit
foo_fit = Foo_fit()

A,B = foo_fit.fit_AB(carbon_number,molar_fraction)
L,M = foo_fit.fit_LM(carbon_number, density)
Ac, Bc = foo_fit.fit_AcBc(carbon_number, molar_fraction)
Ad = foo_fit.fit_Ad(carbon_number, density)[0]
C = foo_fit.fit_C(carbon_number,molecular_weight) [0]

fit_parameters = {
    'A':A, 'B':B, 'L':L, 'M':M, 'Ac':Ac, 'Bc':Bc
    ,'Ad':Ad,'C':C
}

fit_parameters

{'A': 20.24141696010174,
 'B': -5.350895658855276,
 'L': 0.5181480391330261,
 'M': 0.11698833413234153,
 'Ac': 2.9969050222015183,
 'Bc': -87.60869574376241,
 'Ad': -0.4578051749233114,
 'C': 13.913346938986416}

Para no utilizar los valores establecidos aleatoriamente por Pedersen para el número máximo de carbono
calculo el valor máximo con el criterio de Cismondi et. al. Este método está definido en la librería Residual_fraction.

In [5]:
from recore import Residual_fraction

residual_fraction = Residual_fraction(molecular_weight_max,molar_fraction_max)

carbon_range_it = np.array(range(carbonnumber_max,201,1))
res_carbon_number_mx = residual_fraction.carbon_number_max(
    carbon_range_it, A, B, C
    )

print('Número máximo de carbono calculado:', res_carbon_number_mx)

Número máximo de carbono calculado: 31


Aplico las funciones de distribución para obtener todas las propiedades extendidas del fluido que se requieren para ingresar luego a la EoS. 

Se realiza para el criterio de Pedersen y para las modificaciones establecidas por Cismondi, cada uno con su correspondiente librería: Distribution_pedersen y Distribution_cismondi.

In [6]:
from recore import Distribution_pedersen, Distribution_cismondi

distribution_pedersen = Distribution_pedersen()
distribution_cismondi = Distribution_cismondi()

c_range = np.arange(carbonnumber_max+1, 81)
ped_density = distribution_pedersen.p_density(c_range,L,M)
cis_density = distribution_cismondi.c_density(c_range, Ad)

ped_mw = distribution_pedersen.p_molecular_weight(c_range)
cis_mw = distribution_cismondi.c_molecular_weight(c_range,C)

ped_mf = distribution_pedersen.p_molar_fraction(c_range,A,B)
cis_mf = distribution_cismondi.c_molar_fraction(c_range, Ac, Bc)

Con este procedimiento ya resolvimos la caracterización de los datos distribuidos. ¿Qué logramos? Distribuir la fracción plus de la muestra de petroleo (aquella fracción que no podemos caracterizar en el laboratorio por la dificultad para separar sus partes), expandir los datos y estamos listos para obtener las propiedades que vamos a utilizar para ingresar a una Ecuación de Estado. 

La distribución para los datos analizados sólo nos permitió expandir óptimamente 3 números de carbono, esto tiene que ver con el criterio de corte en donde establezco que se detenga la distribución cuando se cumple el balance de masa. Esto debe ajustarse con una metodología planteada en el paper de cismondi et. al. y la flexibilidad del paquete me permite realizarlo (es una tarea en la que estoy trabajando que luego modificará la función carbon_number_max)

In [7]:
df ['molar_percentage_pedersen'] = df ['molar_percentage']
df ['molar_percentage_cismondi'] = df ['molar_percentage']
df ['molecular_weight_pedersen'] = df ['molar_mass']
df ['molecular_weight_cismondi'] = df ['molar_mass']
df ['density_pedersen'] = df ['density']
df ['density_cismondi'] = df ['density']

df = df.drop(['molar_mass','molar_percentage','density'],  axis=1)

In [8]:
df['Tc_pedersen'] = np.nan
df['Tc_cismondi'] = np.nan
df['Pc_pedersen'] = np.nan
df['Pc_cismondi'] = np.nan

### Correlaciones para determinar Tc, Pc, factor acéntrico etc. 

In [9]:
from recore import Correlations

correlations = Correlations('PR')

tc_ped = correlations.critical_temperature(ped_mw,ped_density)
tc_cis = correlations.critical_temperature(cis_mw,cis_density)
pc_ped = correlations.critical_pression(ped_mw, ped_density)
pc_cis = correlations.critical_pression(cis_mw,cis_density)

In [10]:
for i in range (len(c_range)):
    df=pd.concat([df,
        pd.DataFrame(
        {'component': c_range[i],
        'molar_percentage_pedersen': ped_mf[i],
        'molar_percentage_cismondi' : cis_mf[i],
        'molecular_weight_pedersen': ped_mw[i],
        'molecular_weight_cismondi': cis_mw[i],
        'density_pedersen': ped_density[i],
        'density_cismondi': cis_density[i], 
        'Tc_pedersen': tc_ped[i],
        'Tc_cismondi': tc_cis[i],
        'Pc_pedersen': pc_ped[i],
        'Pc_cismondi': pc_cis[i]   
        }, index = [c_range[i]])])
df

,component,molar_percentage_pedersen,molar_percentage_cismondi,molecular_weight_pedersen,molecular_weight_cismondi,density_pedersen,density_cismondi,Tc_pedersen,Tc_cismondi,Pc_pedersen,Pc_cismondi
0,7,2.870000,2.870000e+00,96.0,96.000000,0.738000,0.738000,NaN,NaN,NaN,NaN
1,8,4.080000,4.080000e+00,107.0,107.000000,0.765000,0.765000,NaN,NaN,NaN,NaN
2,9,3.510000,3.510000e+00,121.0,121.000000,0.781000,0.781000,NaN,NaN,NaN,NaN
3,10,3.260000,3.260000e+00,134.0,134.000000,0.792000,0.792000,NaN,NaN,NaN,NaN
4,11,2.510000,2.510000e+00,147.0,147.000000,0.796000,0.796000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
76,76,0.000030,7.396304e+60,1060.0,1057.934286,1.024793,0.936020,1407.306508,1399.318298,70.818005,1.902461
77,77,0.000025,1.480997e+62,1074.0,1071.847633,1.026323,0.936042,1417.383869,1409.225937,82.392828,1.899279
78,78,0.000021,2.965469e+63,1088.0,1085.760980,1.027832,0.936061,1427.442585,1419.116372,96.199763,1.896150
79,79,0.000017,5.937897e+64,1102.0,1099.674327,1.029322,0.936079,1437.483143,1428.990059,112.721712,1.893073


In [11]:
from recore import Lumping

lumping = Lumping()
dat = lumping.lumpy([(7,11),(12,14),(15,20),(21,30),(31,40),(41,50),(51,80)],df,'component')


/home/marlen/Charapy/Charapy/recore.py:545: FutureWarning: Interpreting call
	'.reindex(a, b)' as 
	'.reindex(index=a, columns=b)'.
Use named arguments to remove any ambiguity. In the future, using positional arguments for 'index' or 'columns' will raise a 'TypeError'.
  datos.reindex(count,'ID')


TypeError: Index(...) must be called with a collection of some kind, 'ID' was passed

# Checkear seaborn para plots.